In [ ]:
import os
from fitting_code import FitModel
import fitting_output as output
from scipy import optimize
import numpy as np
from input_checker import setup_error_checks

In [ ]:
def random_set_of_structures(fits, structures, structures_to_fit):
    """
    Randomly selects structures up to the number of structures to include in the fit, checks there are no repeats and returns the structure numbers of those to be included in the fit.
    Args:
        fits (int): Number of fits to run.
        structures (int): Total number of structures in the training set.
        structures_to_fit (int): Number of structures to fit to.
    Returns:
        (np.array): Structure numbers of structures in training set to be fitted to.
    """
    sets_of_structures = []
    while len(sets_of_structures) < fits:
        struct_set = np.sort(np.random.randint(0, structures, size=structures_to_fit), axis=0)
        if len(set(struct_set)) != structures_to_fit:
            continue
        if not any(np.array_equiv(struct_set, x) for x in sets_of_structures):
            sets_of_structures.append(struct_set) 
    return np.array(sets_of_structures)

def run_fit(sets_of_structures, params, labels, bounds, supercell=None):
    """
    Collates the structures to be fitted into the working directory, creates the lammps inputs and runs the optimiser to fit to the designated parameters. Calls another function to save the data in the appropriate output directory.
    Args:
        sets_of_structures (np.array): Structure numbers of structures in training set to be fitted to.
        params (dict(dict)): Setup dictionary containing the inputs for coreshell, charges, masses, potentials, and core-shell springs.
        labels (list(str)): List of parameters to be fitted.
        bounds (list(tuple(float))): List of lower and upper bound tuples associated with each parameter.
        supercell (optional:list(int) or list(list(int))): 3 integers defining the cell increase in x, y, and z for all structures, or a list of lists where each list is 3 integers defining the cell increase in x, y, z, for each individual structure in the fitting process. i.e. all increase by the same amount, or each structure increased but different amounts. Default=None.
    Returns:
        None
    """ 
    poscars = os.path.join('poscars','training_set')
    outcars = os.path.join('outcars','training_set')
    for fit, structs in enumerate(sets_of_structures): 
        for struct_num, struct in enumerate(structs):
            os.system('cp {}/POSCAR{} {}/POSCAR{}'.format(poscars, struct+1, 'poscars', struct_num+1))
            os.system('cp {}/OUTCAR{} {}/OUTCAR{}'.format(outcars, struct+1, 'outcars', struct_num+1))
        fit_data = FitModel.collect_info(params, supercell=supercell)
        setup_error_checks(labels, bounds, fit_data, params)
        fit_output = optimize.differential_evolution(fit_data.chi_squared_error, bounds=bounds, popsize=25,
                                            args=([labels]), maxiter=1,
                                            disp=True, init='latinhypercube', workers=-1)
        dft_forces, ip_forces, dft_stresses, ip_stresses = output.extract_stresses_and_forces(fit_data, fit_output.x, labels)
        local_struct_dir = '-'.join([ '{}'.format(struct+1) for struct in structs])
        struct_directory = output.create_directory(head_directory_name, local_struct_dir)
        output.save_data(struct_directory, labels, fit_output, dft_forces, ip_forces, dft_stresses, ip_stresses)

## Setup of fitting parameters for your structure (Example: LiNiO2)
Params is the dictionary (of dictionaries) which contains the main information relating to the system and potentials. There are 5 sub dictionaries: core_shell, charges, masses, potentials, and cs_springs.

**core_shell**: The keys are each atom type within the structure, with the relating value a boolean expression stating if that atom type is core-shell or not i.e. True = core-shell, False = rigid ion.

**charges**: The keys are again each atom type within the structure. The relating value is either a float representation of the atomic charge (for rigid ion atoms) or a sub dictionary where the sub keys are 'core' and 'shell', with the relating sub values being a float representation of the charge. Note: if you are fitting the charge separation (dq), the formal charge should be on the core and 0.0 charge on the shell.

**masses**: Similar to charges, the keys are each atom type within the structure, with the values either a float representation of the atomic mass, or a sub directory with the sub keys 'core' and 'shell' and sub values a float representation of the mass on each (summing to the atomic mass). Mass can not be fitted, and there is no definative way of splitting the mass, however convention suggests having 10 % mass on the shell.

**potentials**: The keys are atom label pairs separated by a dash (str), example: 'Li-O'. The values are a list of the buckingham potential parameters, i.e. [a, rho, c], where each parameter is a float.

**cs_springs**: The keys are agian atom label pairs separated by a dash (str), example: 'O-O'. This basically denotes the spring is between 'O' core and 'O' shell. The values are a list of the spring contants, k1 and k2, as floats. Commonly k2 is set to 0.0.

In [ ]:
params = {}
params['core_shell'] = { 'Li': False, 'Ni': False, 'O': True }
params['charges'] = {'Li': +1.0,
                     'Ni': +3.0,
                     'O': {'core':  -2.0,
                           'shell': 0.0}} 
params['masses'] = {'Li': 6.941,
                    'Ni': 58.6934,
                    'O': {'core': 14.3991,
                          'shell': 1.5999} }
params['potentials'] = {'Li-O': [663.111, 0.119, 0.0],
                        'Ni-O': [1393.540, 0.218, 0.000],
                        'O-O': [25804.807, 0.284, 0.0]}
params['cs_springs'] = {'O-O' : [20.0, 0.0]}

Labels and bounds relate to the parameters you wish to include in your fit. Labels is a list of strings relating to the individual parameter. Accepted keys are:

**dq_x**: Where x is the atomic symbol of the core-shell atom i.e. 'dq_O'. This parameter relates to the charge separation between the core and the shell, where dq is added to the charge for the core, and subtracted from the charge of the shell. In params, the charge should be split with the full charge on the core and 0.0 charge on the shell when this parameter is fitted.

**q_scaling**: This parameter allows you to fit partial charges, where the term itself is the scaling factor. i.e. a q_scaling value of 0.5 is 0.5 x formal charge.

**x-x spring**: Where x is the atomic symbol of the core-shell atom the spring constant relates to. Example: 'O-O spring'.

**x_y_param**: Where x and y are the atoms associated with the potential parameter 'param'. Param can be a, rho, or c only. Examples: 'Li_O_a', 'Li_O_rho', and 'Li_O_c'. Note: DFT doesn't use a dispersion term, which is why the 'c' parameter is not fitted in this example.

Bounds are a list of tuples containing the upper and lower values, i.e. bounds, you wish to impose on the relating parameter. Shown here the parameter labeled 'dq_O' has a lower bound of 0.001 and an upper bound of 4. The bounds position in the list must directly corespond to the label in the labels list.

In [ ]:
labels = ['dq_O', 'q_scaling', 'O-O spring', 'Li_O_a', 'Li_O_rho', 'Ni_O_a', 'Ni_O_rho', 'O_O_a', 'O_O_rho']
bounds = [(0.01, 4), (0.3,1.0), (10.0,150.0), (100.0,50000.0), (0.01,1.0), (100.0,50000.0), (0.01,1.0), (150.0,50000.0), (0.01,1.0)]

Below is an example input of params, lables, and bounds for a **fully rigid ion** system.
Note: The only non-required params key is 'cs_springs' as no spring constant used.

In [ ]:
# params = {}
# params['core_shell'] = { 'Li': False, 'Ni': False, 'O': False }
# params['charges'] = {'Li': +1.0,
#                      'Ni': +3.0,
#                      'O': -2.0}
# params['masses'] = {'Li': 6.941,
#                     'Ni': 58.6934,
#                     'O': 15.999}
# params['potentials'] = {'Li-O': [663.111, 0.119, 0.0],
#                         'Ni-O': [1393.540, 0.218, 0.000],
#                         'O-O': [25804.807, 0.284, 0.0]}

# labels = ['q_scaling', 'Li_O_a', 'Li_O_rho', 'Ni_O_a', 'Ni_O_rho', 'O_O_a', 'O_O_rho']
# bounds = [(0.3,1.0), (100.0,50000.0), (0.01,1.0), (100.0,50000.0), (0.01,1.0), (150.0,50000.0), (0.01,1.0)]

## Directory set up

Structures, structures_to_fit, and fits are required inputs. These designate how many structures are in the training set (saved in a sub directory named 'training_set' in both the 'poscars' and 'outcars' directories), how many of those structures to fit to, and how many fits to conduct, respectively.

The 'head_directory_name' is set up to output the fit to a results directory, with a sub directory of the number of structures fitted. For example if you fit to 5 structures, your output would go to 'results/5_fit'. This can be changed without causing any issues.

In [ ]:
structures = 15 #Total number of structures in the training set
structures_to_fit = 1 #Number of structures you wish to fit to
fits = 1 #Number of fits to run
head_directory_name = 'results/{}_fit'.format(structures_to_fit)

## Randomly selecting sets of structures
To prevent any bias in the data, and to improve the variation in structures with are beeing fitted to, structures are selected randomly from the trianing set up to the number of structures you are decided to fit to. This is done in a function which will check that repeats of the same structure aren't used and doesn't require an input further to that given above. Note: training set must be saved in the correct place to run properly, otherwise errors will arise.

In [ ]:
sets_of_structures = random_set_of_structures(fits, structures, structures_to_fit)

## Running the fit for each set of structures
The fit itself is run within the function above called 'run_fit', which is executed here. There are 4 required inputs: 'sets_of_structures', 'params', 'labels', and 'bounds', which have already been defined. There is also 1 optional input, 'supercell' which creates a supercell of the structures before running the fit. This can either be a list with the multipliers for x, y, and z, i.e. [2,2,2], or a list of list if you want to use different system sizes (not recommended) or have different starting sizes, i.e. [[2,2,2],[1,1,2]].

If you want to change any of the optimisation parameters, such as the population size or initiation method, you need to do this within the *run_fit* function. The fitting process uses scipy.optimise.differential_evolution of which the documentation and input parameters can be found at https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html.

**Note**: workers relates to paralising the fit. It is currently set to '-1' which uses all avaliable cores. This will need to be changed appropriately if used on a shared machine.

Output data is sent to the *head directory* location, in a sub directory named with the structure numbers which it has been fitted to. Example: fitting to 3 random structures will be in a sub directory similar to '2-4-10'.

In [ ]:
run_fit(sets_of_structures, params, labels, bounds)#, supercell=[2,2,2])

## Other notes

changing cut offs --> in lammps_data.py


make clean function after running the fit to remove the outcars/poscars from the directory
set up the lammps directory automatically on clone.